# Network Analysis with Emissions

## Example from social circles data


### Data Description

https://snap.stanford.edu/data/ego-Facebook.html

This dataset (*facebook_combined.txt*) consists of circles from Facebook (4,039 nodes and 88,234 edges) 

### Load required libraries

In [38]:
import networkx as nx
import scipy.sparse as sp
import psutil, time, os, gc, statistics, warnings
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
from collections import defaultdict, deque
from tqdm import tqdm

warnings.filterwarnings('ignore')

### Functions

In [ ]:
# Function to get memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Memory in MB

# Function to run analysis with multiple iterations
def run_analysis_with_repeats(analysis_func, file_path, num_runs=5):
    results = []
    emissions = []
    
    for _ in range(num_runs):
        gc.collect() # clear memory before each run
        tracker = EmissionsTracker()
        tracker.start()
        result = analysis_func(file_path)
        emissions.append(tracker.stop())
        results.append(result)
    
    # Compute statistics
    metrics = {
        "Method": results[0]["Method"],
        "Nodes": results[0]["Nodes"],
        "Edges": results[0]["Edges"],
        "Avg Degree": results[0]["Avg Degree"],
        "Density": results[0]["Density"],
        "Longest Path": results[0]["Longest Path"],
        "Longest Path Nodes": results[0]["Longest Path Nodes"],
        "Shortest Path": results[0]["Shortest Path"],
        "Shortest Path Nodes": results[0]["Shortest Path Nodes"],
        "Time (s)": statistics.mean([r["Time (s)"] for r in results]),
        "Time Std (s)": statistics.stdev([r["Time (s)"] for r in results]) if num_runs > 1 else 0,
        "Memory (MB)": statistics.mean([r["Memory (MB)"] for r in results]),
        "Memory Std (MB)": statistics.stdev([r["Memory (MB)"] for r in results]) if num_runs > 1 else 0,
        "Emissions (kgCO2)": statistics.mean(emissions),
        "Emissions Std (kgCO2)": statistics.stdev(emissions) if num_runs > 1 else 0
    }
    
    return metrics, results[0]["Top 20 Nodes"]  # Return metrics and top nodes from first run

# Naive implementation using adjacency list
def analyze_naive(file_path):
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Build adjacency list
    graph = defaultdict(set)
    with open(file_path, 'r') as f:
        for line in f:
            n1, n2 = map(int, line.strip().split())
            graph[n1].add(n2)
            graph[n2].add(n1)
    
    num_nodes = len(graph)
    num_edges = sum(len(neighbors) for neighbors in graph.values()) // 2
    avg_degree = sum(len(neighbors) for neighbors in graph.values()) / num_nodes if num_nodes > 0 else 0
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Simple BFS for shortest path and tracking nodes
    def bfs_shortest_path(start, end):
        if start == end:
            return 0, [start]
        visited = {start}
        queue = [(start, [start])]
        while queue:
            node, path = queue.pop(0)
            for neighbor in graph[node]:
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    if neighbor == end:
                        return len(new_path) - 1, new_path
                    visited.add(neighbor)
                    queue.append((neighbor, new_path))
        return float('inf'), []
    
    # Find approximate longest and shortest paths
    max_path = 0
    min_path = float('inf')
    max_path_nodes = []
    min_path_nodes = []
    sample_nodes = list(graph.keys())[:min(100, num_nodes)]  # Sample for efficiency
    for i, start in enumerate(sample_nodes):
        for end in sample_nodes[i+1:]:
            path_len, path_nodes = bfs_shortest_path(start, end)
            if path_len != float('inf'):
                if path_len > max_path:
                    max_path = path_len
                    max_path_nodes = path_nodes
                if path_len > 0 and path_len < min_path:
                    min_path = path_len
                    min_path_nodes = path_nodes
    
    # Find top 20 nodes by degree
    degrees = [(node, len(neighbors)) for node, neighbors in graph.items()]
    top_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Longest Path: {max_path}")
    print(f"Longest Path Nodes: {max_path_nodes}")
    print(f"Shortest Path: {min_path if min_path != float('inf') else 'N/A'}")
    print(f"Shortest Path Nodes: {min_path_nodes if min_path != float('inf') else 'N/A'}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "Naive (Adjacency List)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Longest Path": max_path,
        "Longest Path Nodes": max_path_nodes,
        "Shortest Path": min_path if min_path != float('inf') else 0,
        "Shortest Path Nodes": min_path_nodes if min_path != float('inf') else [],
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Top 20 Nodes": top_nodes_dict
    }

# Function to analyze full graph with NetworkX
def analyze_networkx(file_path):
    start_time = time.time() 
    start_memory = get_memory_usage()
    
    G = nx.read_edgelist(file_path, nodetype=int, create_using=nx.Graph())
    
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Find longest and shortest paths with nodes
    longest_path = 0
    shortest_path = float('inf')
    longest_path_nodes = []
    shortest_path_nodes = []
    if num_nodes > 1:
        for path in nx.all_pairs_shortest_path(G):
            path_len = len(path[1][list(path[1].keys())[-1]]) - 1
            if path_len > longest_path:
                longest_path = path_len
                longest_path_nodes = path[1][list(path[1].keys())[-1]]
            if path_len > 0 and path_len < shortest_path:
                shortest_path = path_len
                shortest_path_nodes = path[1][list(path[1].keys())[-1]]
    
    # Find top 20 nodes by degree
    degrees = G.degree()
    top_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Longest Path: {longest_path}")
    print(f"Longest Path Nodes: {longest_path_nodes}")
    print(f"Shortest Path: {shortest_path if shortest_path != float('inf') else 'N/A'}")
    print(f"Shortest Path Nodes: {shortest_path_nodes if shortest_path != float('inf') else 'N/A'}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "NetworkX (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Longest Path": longest_path,
        "Longest Path Nodes": longest_path_nodes,
        "Shortest Path": shortest_path,
        "Shortest Path Nodes": shortest_path_nodes,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Top 20 Nodes": top_nodes_dict
    }

# SciPy sparse matrix analysis
def analyze_scipy_sparse(file_path):
    start_time = time.time()   
    start_memory = get_memory_usage()
    
    # Load edge list with pandas for better performance
    edges_df = pd.read_csv(file_path, sep=' ', header=None, dtype=np.int32, engine='c')
    edges = edges_df.to_numpy()
    
    # Vectorized node mapping
    nodes, inverse_indices = np.unique(edges, return_inverse=True)
    num_nodes = len(nodes)
    edge_indices = inverse_indices.reshape(edges.shape)  # Shape: (m, 2)
    
    # Create row and column arrays for symmetric adjacency matrix
    rows = np.concatenate([edge_indices[:, 0], edge_indices[:, 1]])
    cols = np.concatenate([edge_indices[:, 1], edge_indices[:, 0]])
    data = np.ones(len(rows), dtype=np.int32)
    
    adj_matrix = sp.csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    
    num_edges = adj_matrix.nnz // 2
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    avg_degree = degrees.mean()
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Convert to NetworkX for path calculations
    G = nx.from_scipy_sparse_array(adj_matrix)
    longest_path = 0
    shortest_path = float('inf')
    longest_path_nodes = []
    shortest_path_nodes = []
    if num_nodes > 1:
        for path in nx.all_pairs_shortest_path(G):
            path_len = len(path[1][list(path[1].keys())[-1]]) - 1
            if path_len > longest_path:
                longest_path = path_len
                longest_path_nodes = path[1][list(path[1].keys())[-1]]
            if path_len > 0 and path_len < shortest_path:
                shortest_path = path_len
                shortest_path_nodes = path[1][list(path[1].keys())[-1]]
    
    # Find top 20 nodes by degree
    top_indices = np.argpartition(degrees, -20)[-20:]
    top_degrees = degrees[top_indices]
    top_nodes = [(nodes[i], degrees[i]) for i in top_indices]
    top_nodes = sorted(top_nodes, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Longest Path: {longest_path}")
    print(f"Longest Path Nodes: {longest_path_nodes}")
    print(f"Shortest Path: {shortest_path if shortest_path != float('inf') else 'N/A'}")
    print(f"Shortest Path Nodes: {shortest_path_nodes if shortest_path != float('inf') else 'N/A'}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "SciPy Sparse (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Longest Path": longest_path,
        "Longest Path Nodes": longest_path_nodes,
        "Shortest Path": shortest_path,
        "Shortest Path Nodes": shortest_path_nodes,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Top 20 Nodes": top_nodes_dict
    }

# Function to run all analyses
def run_all_analyses(file_path, num_runs):
    methods = [
        ("Naive", analyze_naive),
        ("NetworkX", analyze_networkx),
        ("SciPy Sparse", analyze_scipy_sparse)
    ]
    results = []
    top_nodes_dfs = []
    
    for method_name, method_func in methods:
        print(f"\nRunning {method_name} analysis ({num_runs} runs)")
        metrics, top_nodes = run_analysis_with_repeats(method_func, file_path, num_runs)
        results.append(metrics)
        top_nodes_df = pd.DataFrame(top_nodes)
        top_nodes_df['Method'] = f"{method_name} (Full)"
        top_nodes_dfs.append(top_nodes_df)
    
    return results, top_nodes_dfs

### Main function and combined output

In [ ]:
# Main execution
def main():
    file_path = "data/facebook_combined.txt"
    
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found. Please download from http://snap.stanford.edu/data/egnets-Facebook.html")
        return
    
    num_runs = 50
    results, top_nodes_dfs = run_all_analyses(file_path, num_runs)
    
    # Combine results
    print("\nTable: Performance Comparison (Averaged over runs)")
    df = pd.DataFrame(results)
    print(df.round(6))
    df.to_csv("01-02-graph_analysis_comparison.csv", index=False)
    print("\nResults saved to 01-02-graph_analysis_comparison.csv")
    
    # Save top nodes to CSV
    top_nodes_combined = pd.concat(top_nodes_dfs, ignore_index=True)
    top_nodes_combined.to_csv("01-02-top_20_nodes.csv", index=False)
    print("Top 20 nodes saved to 01-02-top_20_nodes.csv")

if __name__ == "__main__":
    main()

Running Analyses:   0%|          | 0/3 [00:00<?, ?run/s][codecarbon WARNING @ 20:19:58] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 20:19:58] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 20:19:58] [setup] RAM Tracking...
[codecarbon INFO @ 20:19:58] [setup] CPU Tracking...



Running Naive analysis (1 runs)


[codecarbon WARNING @ 20:19:58] We saw that you have a Apple M4 but we don't know it. Please contact us.
[codecarbon WARNING @ 20:19:58] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 20:19:58] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 20:19:58] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 20:19:58] [setup] GPU Tracking...
[codecarbon INFO @ 20:19:58] No GPU found.
[codecarbon INFO @ 20:19:58] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 20:19:58] >>> Tracker's metadata:
[codecarbon INFO @ 20:19:58]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 20:19:58]   P

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Longest Path Length: 2
Longest Path Weight: 2.00
Shortest Path Length: 1
Shortest Path Weight: 1.0
Execution Time: 0.10 seconds
Memory Used: 4.33 MB
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220

Running NetworkX analysis (1 runs)


[codecarbon WARNING @ 20:20:02] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 20:20:02] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 20:20:02] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 20:20:02] [setup] GPU Tracking...
[codecarbon INFO @ 20:20:02] No GPU found.
[codecarbon INFO @ 20:20:02] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 20:20:02] >>> Tracker's metadata:
[codecarbon INFO @ 20:20:02]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 20:20:02]   Python version: 3.13.5
[codecarbon INFO @ 20:20:02]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 20:20:0

KeyboardInterrupt: 

[codecarbon INFO @ 20:20:16] Energy consumed for RAM : 0.013492 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 20:20:16] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 20:20:16] Energy consumed for All CPU : 0.191819 kWh
[codecarbon INFO @ 20:20:16] 0.205311 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:20:18] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 20:20:18] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 20:20:18] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 20:20:18] 0.000190 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:20:20] Energy consumed for RAM : 0.013603 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 20:20:20] Delta energy consumed for CPU with constant : 0.000175 kWh, power : 42.5 W
[codecarbon INFO @ 20:20:20] Energy consumed for All CPU : 0.193424 kWh
[codecarbon INFO @ 20:20:20] 0.207027 kWh of 

## Reference

1. 